In [1]:
# Add path candidates to sys.path to import utility / helper modules
import sys
sys.path.append('/workspace')

In [2]:
%load_ext pycodestyle_magic
%pycodestyle_on

In [3]:
"""
Version 1:
- used methods: DP

"""
from functools import lru_cache

INF = float('inf')


def get_overlapping_length(a, b):
    """
    Returns the length of non-overlapping partial of :a
    where satisfies :b.startswith(:a[length:])
    """
    for i in range(len(a)):
        suffix = a[i:]
        if b.startswith(suffix):
            return i

    return len(a)


# Solution
def solution(substrs):
    # Excludes all substrings which is part of anohter substring
    excluded_substrs = set()
    for a in substrs:
        for b in substrs:
            if a == b:
                continue
            elif a in b:
                excluded_substrs.add(a)

    # Index 0 is used as dummy (-1)
    substrs = ['## DUMMY ##'] \
              + sorted([s for s in substrs if s not in excluded_substrs])  # noqa: E127
    n = len(substrs)
    all_taken = (1 << n) - 1

    # Pre-calculate non-overlapping for each possible pair of substrings
    # substrs[0] is dummy string, and is ignored because its length is set to 0
    overlapping_length = [
        [0 if i == 0 else get_overlapping_length(substrs[i], substrs[j])
         for j in range(n)] for i in range(n)
    ]

    # Remember the choices made for each step for reconstruction
    choices = [
        [0 for _ in range(1 << n)] for _ in range(n)
    ]

    # Calculate cost and path for reconstruction
    @lru_cache(maxsize=n*(1 << n))
    def solve(prev, taken):
        if taken == all_taken:
            return len(substrs[prev])

        ret = INF
        for next in range(1, n):
            if prev == 0 or not bool(taken & (1 << next)):
                temp = overlapping_length[prev][next] \
                     + solve(next, taken | (1 << next))
                if temp < ret:
                    ret = temp
                    choices[prev][taken] = next

        return ret

    # Reconstruct the string from choices. Must be called
    # after solve is called.
    def reconstruct(prev, taken):
        if taken == all_taken:
            return substrs[prev]

        next = choices[prev][taken]
        length = overlapping_length[prev][next]
        return substrs[prev][:length] + reconstruct(next, taken | (1 << next))

    # Argument taken should be 1 (dummy string)
    _cost = solve(0, 1)
    result = reconstruct(0, 1)
    return result


# Main I/O part
def main(rl):
    C = int(rl())
    for _ in range(C):
        k = int(rl())
        substrs = []
        for _ in range(k):
            substrs.append(rl().strip())

        result = solution(substrs)
        print(result)


# Additional codes to simulate I/O
try:
    import IPython
except ImportError as _:
    # Submit env
    import sys
    main(sys.stdin.readline)
else:
    from helpers import runner
    # IPython env
    runner.run(main)

In [4]:
"""
Testing

"""
from helpers import testing


def epsilon(a, b):
    return abs(b - a) < 10e-8


test_cases = [
    {
        'input': [('geo', 'oji', 'jing')],
        'expected': 'geojing',
    },
    {
        'input': [('world', 'hello')],
        'expected': 'helloworld',
    },
    {
        'input': [('abrac', 'cadabra', 'dabr')],
        'expected': 'cadabrac',
    },
    {
        'input': [('aaa', 'aaa')],
        'expected': 'aaa',
    },
    {
        'input': [('cabbac', 'bb', 'abb')],
        'expected': 'cabbac',
    },
    {
        'input': [('ababa', 'babab', 'ababc')],
        'expected': 'abababc',
    },
    {
        'input': [('aaaa', 'aaa', 'aa')],
        'expected': 'aaaa',
    },
    {
        'input': [tuple(c * 15 for c in 'abcdefghijklmno')],
        'expected': 'abcdefghijklmno',
        'oracle': lambda a, b: set(a) == set(b),
        'verbose': True,
        'timeout': 60,
    },
]

testing.run(solution, test_cases)

[0.00018820] solution(('geo', 'oji', 'jing')) → 'geojing'
Case 1 PASS

[0.00051300] solution(('world', 'hello')) → 'helloworld'
Case 2 PASS

[0.00008520] solution(('abrac', 'cadabra', 'dabr')) → 'cadabrac'
Case 3 PASS

[0.00003780] solution(('aaa', 'aaa')) → 'aaa'
Case 4 PASS

[0.00002310] solution(('cabbac', 'bb', 'abb')) → 'cabbac'
Case 5 PASS

[0.00018910] solution(('ababa', 'babab', 'ababc')) → 'abababc'
Case 6 PASS

[0.00002930] solution(('aaaa', 'aaa', 'aa')) → 'aaaa'
Case 7 PASS

[2.21490440] solution(('aaaaaaaaaaaaaaa', 'bbbbbbbbbbbbbbb', ') → 'aaaaaaaaaaaaaaabbbbbbbbbbbbbbbcccccccccccccccdddddddddddddddeeeeeeeeeeeeeeeffff
Case 8 PASS

         249943 function calls (4168 primitive calls) in 2.269 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      240    0.001    0.000    0.002    0.000 <ipython-input-3-7127fe598949>:11(get_overlapping_length)
        1    0.000    0.000    2.269    2.269 <ipython-input-3-7127fe5